In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import matplotlib

import pandas as pd
import numpy as np
import seaborn as sns
import itertools

import sys
sys.path.append('../data/tecantaloupe/')

from growth.plate_spec import PlateSpec
from growth.plate_time_course_parser import SavageLabM1000Excel
from matplotlib import pyplot as plt

import seaborn as sns
from utils import *

matplotlib.rcParams.update({'font.size': 30})

In [17]:
allPfams = pd.read_csv('../data/coocurance/ALL_PFAM.txt',sep='\t')
allPfams

,Query ID,Neighbor ID,Neighbor Pfam,SSN Query Cluster #,SSN Query Cluster Color,Query-Neighbor Distance,Query-Neighbor Directions
0,A0A0Q7I8V6,A0A0Q7I9D6,PF00004,1,#FF0000,13,complement-normal
1,A0A248K2M4,A0A248K2S2,PF00004,1,#FF0000,17,complement-complement
2,A0A074JF46,A0A074JBY5,PF00004,1,#FF0000,8,normal-normal
3,A0A2N6DRE4,A0A2N6DRF5,PF00004,1,#FF0000,9,complement-normal
4,A0A1Y0NE98,A0A1Y0NDR2,PF00004,1,#FF0000,19,normal-complement
5,D4GWQ7,D4GSN1,PF00004,1,#FF0000,17,complement-normal
6,C7RLG2,C7RLH0,PF00004,1,#FF0000,8,normal-normal
7,C1DDM9,C1DDP9,PF00004,1,#FF0000,20,normal-complement
8,A0A1G5HPV2,A0A1G5HP54,PF00004-PF01434,1,#FF0000,17,normal-normal
9,A0A1G5HPV2,A0A1G5HP54,PF00004-PF01434,1,#FF0000,17,normal-normal


In [24]:
hasdabB = allPfams[allPfams['Neighbor Pfam'].str.contains('PF00361')]
print(len(hasdabB))
print(len(hasdabB['Query ID'].unique()))
posOperons = hasdabB[(hasdabB['Query-Neighbor Distance']<=3) & 
                 ((hasdabB['Query-Neighbor Directions'] == "normal-normal") 
                  | (hasdabB['Query-Neighbor Directions'] == 'complement-complement'))]['Query ID'].unique()
print(len(posOperons))
hasdabB

1734
840
835


,Query ID,Neighbor ID,Neighbor Pfam,SSN Query Cluster #,SSN Query Cluster Color,Query-Neighbor Distance,Query-Neighbor Directions
8113,A0A0Q6RAM1,A0A0Q6RMA6,PF00361,1,#FF0000,1,normal-normal
8114,A0A0U5B7A7,A0A0U5BBV8,PF00361,1,#FF0000,1,normal-normal
8115,A0A0R0LHR2,A0A0R0LH82,PF00361,1,#FF0000,1,normal-normal
8116,A0A149VTP0,A0A149VUJ7,PF00361,1,#FF0000,1,normal-normal
8117,A0A0W1SUY6,A0A0W1SUX4,PF00361,1,#FF0000,1,normal-normal
8118,A0A241R5F6,A0A241R6M8,PF00361,1,#FF0000,1,complement-complement
8119,A0A2S0III6,A0A2S0III1,PF00361,1,#FF0000,1,normal-normal
8120,A3WMF2,A3WMF1,PF00361,1,#FF0000,1,normal-normal
8121,A0A0F3K8W1,A0A0F3K915,PF00361,1,#FF0000,1,normal-normal
8122,G2J578,G2J577,PF00361,1,#FF0000,1,normal-normal


In [10]:
dabBcooc = pd.read_csv('../data/coocurance/pfam_neighbors_PF00361.txt',sep='\t')
print(len(dabBcooc))
dabBcooc

1099


,Query ID,Neighbor ID,Neighbor Pfam,SSN Query Cluster #,SSN Query Cluster Color,Query-Neighbor Distance,Query-Neighbor Directions
0,A0A0Q6RAM1,A0A0Q6RMA6,PF00361,1,#FF0000,1,normal-normal
1,A0A0U5B7A7,A0A0U5BBV8,PF00361,1,#FF0000,1,normal-normal
2,A0A0R0LHR2,A0A0R0LH82,PF00361,1,#FF0000,1,normal-normal
3,A0A149VTP0,A0A149VUJ7,PF00361,1,#FF0000,1,normal-normal
4,A0A0W1SUY6,A0A0W1SUX4,PF00361,1,#FF0000,1,normal-normal
5,A0A241R5F6,A0A241R6M8,PF00361,1,#FF0000,1,complement-complement
6,A0A2S0III6,A0A2S0III1,PF00361,1,#FF0000,1,normal-normal
7,A3WMF2,A3WMF1,PF00361,1,#FF0000,1,normal-normal
8,A0A0F3K8W1,A0A0F3K915,PF00361,1,#FF0000,1,normal-normal
9,G2J578,G2J577,PF00361,1,#FF0000,1,normal-normal


In [11]:
closestNeighbors = dabBcooc.loc[dabBcooc.groupby('Query ID')['Query-Neighbor Distance'].idxmin()]
print(len(closestNeighbors))
closestNeighbors

840


,Query ID,Neighbor ID,Neighbor Pfam,SSN Query Cluster #,SSN Query Cluster Color,Query-Neighbor Distance,Query-Neighbor Directions
255,A0A011NU83,A0A011PP86,PF00361,1,#FF0000,1,normal-normal
259,A0A011P2D8,A0A011N839,PF00361,1,#FF0000,1,normal-normal
270,A0A011QKE7,A0A011RE14,PF00361,1,#FF0000,1,normal-normal
877,A0A017T271,A0A017T319,PF00361-PF00662,1,#FF0000,1,normal-normal
190,A0A023DJ21,A0A023DJ92,PF00361,1,#FF0000,1,normal-normal
102,A0A024HFH1,A0A024HEZ5,PF00361,1,#FF0000,1,normal-normal
60,A0A059IQX4,A0A059IQN8,PF00361,1,#FF0000,1,complement-complement
798,A0A061JI62,A0A061JIU9,PF00361-PF00662,1,#FF0000,1,complement-complement
772,A0A063ZIT4,A0A063ZGT8,PF00361-PF00662,1,#FF0000,1,normal-normal
686,A0A068YQL2,A0A068YI24,PF00361-PF00662,1,#FF0000,1,complement-complement


In [12]:
posOperons = closestNeighbors[(closestNeighbors['Query-Neighbor Distance']<=3) & 
                 ((closestNeighbors['Query-Neighbor Directions'] == "normal-normal") 
                  | (closestNeighbors['Query-Neighbor Directions'] == 'complement-complement'))]
len(posOperons)

834

In [13]:
posOperons = closestNeighbors[((closestNeighbors['Query-Neighbor Directions'] == "normal-normal") 
                  | (closestNeighbors['Query-Neighbor Directions'] == 'complement-complement'))]
len(posOperons)

837

In [14]:
posOperons = dabBcooc[(dabBcooc['Query-Neighbor Distance']<=3) & 
                 ((dabBcooc['Query-Neighbor Directions'] == "normal-normal") 
                  | (dabBcooc['Query-Neighbor Directions'] == 'complement-complement'))]['Query ID'].unique()
len(posOperons)

835

In [15]:
rubcooc = pd.read_csv('../data/coocurance/pfam_neighbors_PF00016.txt',sep='\t')
print(len(rubcooc))
print(len(rubcooc['Query ID'].unique()))
rubcooc

48
45


,Query ID,Neighbor ID,Neighbor Pfam,SSN Query Cluster #,SSN Query Cluster Color,Query-Neighbor Distance,Query-Neighbor Directions
0,C6HYI8,C6HYJ2,PF00016,1,#FF0000,4,normal-normal
1,A0A0R0LHR2,A0A0R0LRY0,PF00016-PF02788,1,#FF0000,17,normal-complement
2,A0A149VTP0,A0A149VTQ8,PF00016-PF02788,1,#FF0000,17,normal-complement
3,Q31HC3,Q31HD9,PF00016-PF02788,1,#FF0000,16,normal-normal
4,A0A1N6DK18,A0A1N6DK16,PF00016-PF02788,1,#FF0000,4,normal-normal
5,A0A1N6DK18,A0A1N6DJZ7,PF00016-PF02788,1,#FF0000,1,normal-normal
6,A0A0P9EA54,A0A0P9C2Q3,PF00016-PF02788,1,#FF0000,6,complement-normal
7,A0A0P9EA54,A0A0P9CK02,PF00016-PF02788,1,#FF0000,12,complement-complement
8,A4YQE4,A4YQD3,PF00016-PF02788,1,#FF0000,11,normal-normal
9,A0A149W153,A0A149W114,PF00016-PF02788,1,#FF0000,18,complement-normal


In [16]:
rubcooc = pd.read_csv('../data/coocurance/pfam_neighbors_PF00016.txt',sep='\t')
print(len(rubcooc))
print(len(rubcooc['Query ID'].unique()))
rubcooc

48
45


,Query ID,Neighbor ID,Neighbor Pfam,SSN Query Cluster #,SSN Query Cluster Color,Query-Neighbor Distance,Query-Neighbor Directions
0,C6HYI8,C6HYJ2,PF00016,1,#FF0000,4,normal-normal
1,A0A0R0LHR2,A0A0R0LRY0,PF00016-PF02788,1,#FF0000,17,normal-complement
2,A0A149VTP0,A0A149VTQ8,PF00016-PF02788,1,#FF0000,17,normal-complement
3,Q31HC3,Q31HD9,PF00016-PF02788,1,#FF0000,16,normal-normal
4,A0A1N6DK18,A0A1N6DK16,PF00016-PF02788,1,#FF0000,4,normal-normal
5,A0A1N6DK18,A0A1N6DJZ7,PF00016-PF02788,1,#FF0000,1,normal-normal
6,A0A0P9EA54,A0A0P9C2Q3,PF00016-PF02788,1,#FF0000,6,complement-normal
7,A0A0P9EA54,A0A0P9CK02,PF00016-PF02788,1,#FF0000,12,complement-complement
8,A4YQE4,A4YQD3,PF00016-PF02788,1,#FF0000,11,normal-normal
9,A0A149W153,A0A149W114,PF00016-PF02788,1,#FF0000,18,complement-normal
